In [ ]:
# default_exp retail_price_processor

# Retail Price Analytics

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
from fastcore.script import *

In [ ]:
#export
class Constants:
    DATA_START_INDEX = 27
    VALID_DATA_ROW_LENGTH = 39
    LOCATIONS = {
    8:("sampars","cross roads"), 
    10:("hi-lo","cross roads"),
    12:("mega mart","waterloo road"),
    15:("John R Wong","new kingston"),
    18:("Brooklyn","halfway tree"),
    20:("hi-lo","halfway tree"),
    22:("shopper's fair","washington boulevard"),
    24:("shopper's fair","Duhaney Park"),
    26:("Loshusan","barbican road"),
    29:("hi-lo","barbican road"),
    31:("hi-lo","constant spring"),
    34:("shoppers fair","harbour view"),
    35:("lee's food","red hills road"), 
    36:("price smart","red hills road"),
    37:("mega mart","portmore"),
    38:("hi-lo","portmore")
    }


In [ ]:
def is_row_not_empty(row):
    row = list(filter(lambda item: len(item.replace(" ","").lstrip()) > 0, row))
    return False if len(row) == 0 else True

row = ["  "," ", " "]
is_row_not_empty(row)


False

In [ ]:
#export
def is_row_not_empty(row:list):
        row = list(filter(lambda item: len(str(item).lstrip()) > 0, row))
        return False if len(row) == 0 else True  

def convert_list_to_dict(source:list):
        dct = { idx:val for idx ,val in enumerate(source)}
        return dct      

In [ ]:
#export  
class RetailPricesAnalyzer:
    def __init__(self,raw_data:str):
        self.raw_data = raw_data
        self.report_date =  self.__get_report_date(raw_data)
        self.data = self.format_data(raw_data)

      

    def __get_report_date(self,raw_data):
        return "september_04_2021" 

    
    def format_data(self,raw_data:list):
        data_rows = self.__get_data(raw_data)
        header_row = ["date", "commodity","variety-source","region","store","price"]
        formatted_data = []
        for row in data_rows: 
            # get location
            for key,value in Constants.LOCATIONS.items():
                commodity = row[0]
                variety_source = row[4]
                store, location = value
                price = row[key].replace("\n","")
                store_data = [self.report_date,commodity,variety_source,location,store,price]
                formatted_data.append(store_data)
                

        formatted_data[:0] = [header_row]
        return formatted_data
   
    
    def __get_data(self,raw_data):
        data  = []
        products_list = raw_data[Constants.DATA_START_INDEX:] 
        for row in products_list: 
            row = row.split("|")
            # convert each row to a dictionary
            if len(row) == Constants.VALID_DATA_ROW_LENGTH:
                # print(row)
                if is_row_not_empty(row):
                    row = convert_list_to_dict(row)
                    data.append(row)
             
           
        return data   

    def write_to_csv(self,output_directory):
        import pandas as pd
        import os
        output_dir = f"{output_directory}/{self.report_date}"
        try:
            os.mkdir(output_dir) 
        except OSError as error: 
            print(f"{output_dir} already exists,not recreating")      

 
        df = pd.DataFrame(self.data)
        out = f"{output_dir}/retail_prices_{self.report_date}.csv"
        print(f"writing data to: {out}")
        df.to_csv(out,sep="|" ,index=False, header=False)     


In [ ]:
#export 
def open_file(path:str):
    lines = None
    try:
        with open(path) as file:
            lines = file.readlines()
    except Exception as e:
        print(f"unable to open file:{path}")  
        print(f"the following error occcurred: {e}")      
    return lines

In [ ]:
#export
def process_retail_price_data(input_path:str, output_directory = "."):
    lines = open_file(input_path)
    analyzer = RetailPricesAnalyzer(lines)        
    analyzer.write_to_csv(output_directory)
    print("Success :-)")

In [ ]:
input_path = "../data/raw/retail_01.16.2021.csv"
output_directory = "../data/output"
run(input_path,output_directory)
!ls ../data/output/september_04_2021/

../data/output/september_04_2021 already exists,not recreating
writing data to: ../data/output/september_04_2021/retail_prices_september_04_2021.csv
Success :-)
retail_prices_september_04_2021.csv


In [ ]:
#export
@call_parse
def main(input_path:Param("The urban municipal prices file path", str),
         output_folder:Param("where the converted file should be written", str),
      ):
    "Clean up the input file and generates individual parish farm gate prices in the specified output folder"
    print(f"input path:{input_path}")
    run(input_path, output_folder)